# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark

In [3]:
from pyspark.sql.types import IntegerType, FloatType, DateType, StringType, BooleanType
from pyspark.sql.types import StructField, StructType
from pyspark.sql.window import Window
from pyspark.sql.functions import when, col, desc, asc, count,avg, sum, round,collect_list, lit, concat
import pyspark.sql.functions as F

In [4]:
from constants import MATCHES_FILE_PATH, DELIVERIES_FILE_PATH

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('IPL-Analysis').getOrCreate()

# Read data

## Matches dataset 

In [7]:
match_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("season", StringType(), True),
    StructField("city", StringType(), True),
    StructField("date", DateType(), True),
    StructField("match_type", StringType(), True),
    StructField("player_of_match", StringType(), True),
    StructField("venue", StringType(), True),
    StructField("team1", StringType(), True),
    StructField("team2", StringType(), True),
    StructField("toss_winner", StringType(), True),
    StructField("toss_decision", StringType(), True),
    StructField("winner", StringType(), True),
    StructField("result", StringType(), True),
    StructField("result_margin", IntegerType(), True),
    StructField("target_runs", IntegerType(), True),
    StructField("target_overs", IntegerType(), True),
    StructField("super_over", StringType(), True),
    StructField("method", StringType(), True),
    StructField("umpire1", StringType(), True),
    StructField("umpire2", StringType(), True)
])

In [8]:
matches = spark.read.csv('matches.csv', schema= match_schema, header=True).cache()

In [9]:
matches.show(5)

+------+-------+----------+----------+----------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+------------+----------+------+---------+--------------+
|    id| season|      city|      date|match_type|player_of_match|               venue|               team1|               team2|         toss_winner|toss_decision|              winner| result|result_margin|target_runs|target_overs|super_over|method|  umpire1|       umpire2|
+------+-------+----------+----------+----------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+------------+----------+------+---------+--------------+
|335982|2007/08| Bangalore|2008-04-18|    League|    BB McCullum|M Chinnaswamy Sta...|Royal Challengers...|Kolkata Knight Ri...|Royal Challengers...|        field|Kolkata Knig

## Deliveries Dataset

In [10]:
deliveries = spark.read.csv('deliveries.csv', inferSchema=True, header=True).cache()

In [11]:
deliveries.show(5)

+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|match_id|inning|        batting_team|        bowling_team|over|ball|     batter| bowler|non_striker|batsman_runs|extra_runs|total_runs|extras_type|is_wicket|player_dismissed|dismissal_kind|fielder|
+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   1| SC Ganguly|P Kumar|BB McCullum|           0|         1|         1|    legbyes|        0|              NA|            NA|     NA|
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   2|BB McCullum|P Kumar| SC Ganguly|           0|         0|         0|       NULL|        0|              NA|            NA|     NA|
|  33

In [12]:
deliveries.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- inning: integer (nullable = true)
 |-- batting_team: string (nullable = true)
 |-- bowling_team: string (nullable = true)
 |-- over: integer (nullable = true)
 |-- ball: integer (nullable = true)
 |-- batter: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- batsman_runs: integer (nullable = true)
 |-- extra_runs: integer (nullable = true)
 |-- total_runs: integer (nullable = true)
 |-- extras_type: string (nullable = true)
 |-- is_wicket: integer (nullable = true)
 |-- player_dismissed: string (nullable = true)
 |-- dismissal_kind: string (nullable = true)
 |-- fielder: string (nullable = true)



# Data cleaning

In [15]:
# Filter out rows where result_margin is null and match has no result
matches = matches.filter(
    ~((col('result_margin').isNull()) & (col('result') == 'no result'))
)

# Change the values of super_over from N/Y to boolean 0/1 for clarity
matches = matches.withColumn('super_over', when(col('super_over')=='N', 0).otherwise(1))

# Rename column method to DLS_used                   
matches = matches.withColumnRenamed('method','dls_used')

# Convert values to boolean
matches = matches.withColumn('dls_used', when(col('dls_used')=='NA', 0).otherwise(1))

# Convert old team names to the new rebranded names for data consistency
replacements = {    "Royal Challengers Bengaluru": "Royal Challengers Bangalore",
                    "Delhi Daredevils": "Delhi Capitals",
                    "Kings XI Punjab": "Punjab Kings",
                    "Rising Pune Supergiant": "Rising Pune Supergiants",
                    "Gujarat Lions": "Gujarat Titans",
                }
matches = matches.replace(replacements,['team1','team2','toss_winner'])

In [16]:
matches.describe().show()

+-------+------------------+------------------+-------------+------------------+---------------+--------------------+-------------------+-------------------+-------------------+-------------+-------------------+-------+------------------+------------------+------------------+----------+--------+----------+----------+
|summary|                id|            season|         city|        match_type|player_of_match|               venue|              team1|              team2|        toss_winner|toss_decision|             winner| result|     result_margin|       target_runs|      target_overs|super_over|dls_used|   umpire1|   umpire2|
+-------+------------------+------------------+-------------+------------------+---------------+--------------------+-------------------+-------------------+-------------------+-------------+-------------------+-------+------------------+------------------+------------------+----------+--------+----------+----------+
|  count|              1090|              1

In [17]:
# Save the cleaned data to DBFS
matches.write.mode("overwrite").option("header", "true").csv("matches_cleaned.csv")